# Metrics and its Visualizations  

The purpose of this jupyter notebook is to visualize some surfaces and the impact of its abstract metric on the geometry of it. With this, we aim at improving the imagination and understanding of metrics and its link to the surfaces in a more graphic
and clearer way. In doing so, the notebook will also show the possibilities of symbolic programming applied on the calculations for the metric.

## Contents
1. [Used Modules](#zero)
2. [First Example - The Cylinder](#first) 
3. [Second Example - The Sphere](#second)
4. [Third Example - The Ellipsoid](#third)
5. [Forth Example - Gnocchi](#forth)

<a id = 'zero'></a>
### Used Modules

To do the symbolic calculations, the SymPy-module is imported, which allows exact calculations. Instead of using approximations, SymPy is created to perform analytic calculations, introducing symbols, which are going to be the mathematical variables. It also provides us a lot of functions, e.g. analytic derivatives, analytic integrals, the possibility to evaluate terms, etc.

In [ ]:
import sympy as sym

Furthermore, it will be necessary to use arrays and functions that can be applied to the matrices and vectors. Moreover, they can be utilized to generate the data for the plots which are going to be created. Therefore, it is common to use the NumPy-module.

In [ ]:
import numpy as np

Posterior, we are going to need [elliptic functions](https://en.wikipedia.org/wiki/Elliptic_integral) to solve an integral, which are provided by the SciPy-module. In addition, this module contains a lot of routines for numerical integration, interpolation, optimization and linear algebra. For the following code, two packets are required; the ´Special´-packet for the elliptic functions and the ´Spatial´-packet for the [Delaunay-triangulation](https://en.wikipedia.org/wiki/Delaunay_triangulation), which is useful to plot the surfaces.

In [ ]:
from scipy import spatial, special

Considering the three dimensional representation of the plots, we are going to use K3D because of its good graphical quality. With K3D , it is also possible to manipulate the plot without restarting the entire program, i.e. rotating it or zoom in or out to see more details.

In [ ]:
import k3d

It is also very suitable to see real-time changes in the graphic after
manipulating some parameters of the plot. To do so, you can use ´[ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html)´, which contains a lot of useful possibilities to transfer values by widgets, such as buttons, radio buttons, color picker, textboxes and all kinds of sliders, which we will use later for this purpose.


In [ ]:
from ipywidgets import widgets,interact

<a id = 'first'></a>
## First Example - The Cylinder

<a id = 'cyl'></a>
The first example will be a cylinder. At first, we want to plot the surface of the cylinder and then we are going to do symbolic calculations in order to determine the metric tensor $g_{ij}$. In this effort, we have to choose a parameterization for the cylinder with $\alpha\in[0,2\pi)$ for the angular component and $z\in[0,1]$ for the height, like in cartesian coordinates. In the beginning, we have to discretize the values of $\alpha$ and $z$, because the computer cannot handle continuous space. Therefore, it is going to be very useful to generate a matrix, that also divides a certain length into smaller pieces and gives all permutation of it. The ´mgrid´-function of NumPy does exactly this. In the following, an example is provided to illustrate this matter. Given a rectangle of length $6$ and height of $3$. We can now divide the length in $4$ pieces and the height into $6$ pieces. With "$0:a$" we tell the function, that we want to slice the interval $[0,a]$. With the complex number, it is possible to decide how many values between $0$ and $a$ will be generated. The same is going to happen for $b$.

In [ ]:
a = 6
b = 3
pieces_a = 4
pieces_b = 6
slices = np.mgrid[0:a:1j*pieces_a, 0:b:1j*pieces_b]
slices


Now, we have a matrix of two matrices with four values (´pieces_a´ times) from $0$ to $6$ and as a result we have $6$ times (´pieces_b´ times) as columns. The second matrix contains the $6$ values (´pieces_b´ times) from $0$ to $3$ as columns at the whole result four times (´pieces_a´ times). If the matrix is transposed, we obtain for the first matrix all the possible permutations with the $0$ from the second matrix, then all permutations with $0.6$ and so on. 

In [ ]:
slices.T

If we want a matrix with two columns and all the possible permutations, it can be achieved by reshaping the matrix. Knowing the number of values in the matrix, in this case $48$, gives us the possibility to form it into a $24\times 2$ matrix.

In [ ]:
slices.T.reshape(24, 2)

If we do not want to think about dimensionality, it is possible to use $-1$ instead of $24$. In this case, the function counts the number of elements and then assigns the fitting dimension. 

In [ ]:
slices.T.reshape(-1, 2)

We will now divide the values of $\alpha$ into $60$ instances and the values of $z$ into $10$ using the ´mgrid´-function, like in the small example. After reshaping, we get an array with all combinations of the $\alpha$-values in the first and the $z$-values in the second column.

In [ ]:
dimalpha = 60
dimz = 10
parameters_cylinder = np.mgrid[0:2*np.pi:1j*dimalpha, 0:1:1j*dimz].T.reshape(-1, 2)
alpha = parameters_cylinder[:, 0]
z = parameters_cylinder[:, 1]

To display the surface, it is necessary to create an array of points in the 3d-space with the values $x$, $y$ and $z$, which belong to the surface. Therefore, we use the parameterization of a cylinder with the radius $r$:
$$\left( \begin{array}{l}
	 x \\
	 y \\
	 z \\
	 \end{array}
	 \right)=\left( \begin{array}{l}
	 r\cos(\alpha) \\
	 r\sin(\alpha) \\
	 z \\
	 \end{array}
	 \right)$$
     
To obtain a vector with points $(x,y,z)$ in every line, we have to insert the determined values. For the radius we choose one. 

In [ ]:
r = 1
points_cylinder = np.array([r*np.cos(alpha), 
                            r*np.sin(alpha), 
                            z]).T

While plotting a surface, by using a small amount of points, it is common to establish a triangulation to interpolate the surface between three points with a triangle. Therefore, we use the [Delaunay-triangulation](https://en.wikipedia.org/wiki/Delaunay_triangulation), which we can take directly from the ´scipy.spatial´-module. 

In [ ]:
tri_cylinder = spatial.Delaunay(parameters_cylinder)

In the last step we can now create the plot, adding the triangulated surface to ´plot_cylinder´.

In [ ]:
plot_cylinder = k3d.plot()
plot_cylinder += k3d.mesh(points_cylinder, tri_cylinder.simplices,color=0xffc000)
plot_cylinder.display() 

We are going to use the same structure of code several times again to design various types of surfaces. Thus, it will be useful to create a class for surfaces, called ´Surface2D´. In the ´\__ init__´-method we set the attributes of a surface: the parameters of the surface and the set of points for the later plot. With the arguments of the ´\__ init__´-method, the user can establish the parameters, inserting the lowest and the highest value of the two parameters and the number of instances for each. With the array ´constants´, it is also possible to give a list of constants to change the size of the plot. The next function ´get_points(self)´ is going to be used later, when we define the precise surface, in this case the cylinder. In the ´display(self, wframe=False, opac=1)´-function, we are creating the plot like before. With ´wframe´, we can display the wireframe of the surface (default-value is ´False´), setting its value to ´True´. If we want to change the opacity, we can change its value as well.  

In [ ]:
class Surface2D:
    def __init__(self, p1min, p1max, dim1, p2min, p2max, dim2, constants=[]):
        self.parameters = np.mgrid[p1min:p1max:1j*dim1, p2min:p2max:1j*dim2].T.reshape(-1, 2)
        self.constants = constants
        self.get_points() 
        
    def get_points(self):
        '''set array self.points containing collection of points on the surface

        '''
        raise NotImplementedError
        
    def display(self, wframe=False, opac=1):
        triangulation = spatial.Delaunay(self.parameters)
        plot = k3d.plot()
        points = k3d.mesh(self.points, triangulation.simplices.astype(np.uint32), wireframe=wframe, 
                          opacity=opac, color=0xffc000)
        plot += points
        
        return points

<a id = 'cylclass'></a>
Now, we can create the cylinder-class, which inherits from the ´Surface2D´-class. The attributes are the same as in the ´Surface2D´-class, but we can now become more precise about the parameters, inserting the information about the [parameterization](#cyl). The ´dim´-arguments can be left empty, this way we can change the resolution later. Hereafter, we can build the ´get_points(self)´-function, by repeating the procedure of the [beginning](#cyl), taking ´constants$[0]$´ as radius of the cylinder. In this case, we choose a radius of one by default.  

In [ ]:
class Cylinder(Surface2D):
    def __init__(self, dim1, dim2, constants=[1]):
        super().__init__(0, 2*np.pi, dim1, 0, 1, dim2, constants)
        
    def get_points(self):
        alpha, z = self.parameters.T
        r = self.constants[0]
        self.points = np.array([r*np.cos(alpha),
                                r*np.sin(alpha),
                                z],).T

If we want to plot the cylinder, we can do this easily by creating the object ´cylinder´ with ´dim1=10´ and ´dim2=60´ and then using the ´display´-function of the ´Surface2D´-class. We receive the same result as before. If you want, you can try to change the radius of the cylinder by a list with the radius as sole element.

In [ ]:
cylinder = Cylinder(60, 10)
cylinder.display()

Now, we want to use SymPy to calculate the metric tensor for this surface. Before we can start the symbolic programming, we have to provide input for the computer about which variables are going to be our symbols. For this process, we describe our surface with the variables $\alpha$, $z$ and our radius $r$.

In [ ]:
r, alpha, z = sym.symbols("r alpha z")

Then we implement our parameterization in local coordinates with the matrix routine of the SymPy-module.

In [ ]:
v_cylinder = sym.Matrix([[r*sym.cos(alpha)],
                         [r*sym.sin(alpha)],
                         [z]]) 

To calculate the metric tensor, we need to compute the derivatives $\frac{\partial v_{cylinder}}{\partial\alpha}$ and $\frac{\partial v_{cylinder}}{\partial z}$ with the ´diff(function, variable)´-function of the SymPy-module.

In [ ]:
dalpha = sym.diff(v_cylinder, alpha)
dz = sym.diff(v_cylinder, z)

We can now determine the metric tensor $g_{\mathrm{Cylinder}}$ by the inner product variations of ´dalpha´ and ´dz´, therefore we use the routine $\langle a,b\rangle$ = ´a.dot(b)´.

In [ ]:
g_cylinder = sym.Matrix([[dz.dot(dz), dz.dot(dalpha)], 
                         [dalpha.dot(dz), 
                          dalpha.dot(dalpha)]])
g_cylinder

As you can see in the output, the expression can be simplified. So we use the ´simplify´-function of SymPy to obtain an easier form of the term and display $g_{ij}$.

In [ ]:
g_cylinder = sym.simplify(g_cylinder)
g_cylinder

This is the common metric of a [cylinder](https://en.wikipedia.org/wiki/Metric_tensor). We can also find the line element. To do so, we define the variables $\mathrm{d}\phi$ and $\mathrm{d}z$ as infinitesimal variation of $\phi$ and $z$ and store it as a matrix: $$\mathrm{d}x = \left( \begin{array}{l}
	 \mathrm{d}\phi \\
	 \mathrm{d}z \\
	 \end{array}
	 \right)$$


In [ ]:
dphi, dz = sym.symbols('dphi dz')
dx = sym.Matrix([dphi, dz])
dx

Here, we determine $\mathrm{d}s^2 = \mathrm{d}x^Tg_{ij}\mathrm{d}x$ with ´a.dot(b)´.

In [ ]:
lineelement = dx.dot(g_cylinder*dx)
lineelement

<a id = 'second'></a> 
## Second Example - The Sphere

<a id = 'example'></a>
Hereafter, we want to apply the same procedure on the surface of a sphere with the two parameters $\phi\in[0,2\pi]$ and $\theta\in[0,\pi]$, which represent the angles and the radius $r$. When we know the angle $\phi$ in the $x$-$y$ plane, the polar angle $\theta$ between the zenith direction, the line segment from the origin to the point and the distance from the origin, that is the radius $r$, we can describe points on the surface of a sphere with a constant distance $r$. 
To do so, we are following the same [procedure](#cylclass) as in the first example, to create the plot.
Hence, we have to define a new class for the sphere, with the constants $1$ for the radius and $2\pi$ for the upper limit of the interval of $\phi$, because later this value is going to be changed to produce a half sphere.
We are going to use the parameterization of the sphere, which is displayed below:
$$\left( \begin{array}{c}
	 x \\
	 y \\
	 z \\
	 \end{array}
	 \right)=\left( \begin{array}{c}
	 r\sin(\theta)\cos(\phi) \\
	 r\sin(\theta)\sin(\phi) \\
	 r\cos(\theta) \\
	 \end{array}
	 \right)$$
To get a vector with a point $(x,y,z)$ in every line, we have to transpose the matrix. 

In [ ]:
class Sphere(Surface2D):
    def __init__(self, dim1, dim2, constants=[1,2*np.pi]):
        super().__init__(0, np.pi, dim1, 0, constants[1] , dim2, constants)
        
    def get_points(self):
        r = self.constants[0]
        theta, phi = self.parameters.T
        self.points = np.array([r*np.sin(theta)*np.cos(phi),
                                r*np.sin(theta)*np.sin(phi),
                                r*np.cos(theta)]).T

<a id = "plotsphere"></a> Afterwards, we can plot the surface of the sphere as we have done previously with the cylinder, creating an object of the type ´Sphere´ , in which we use 60 pieces for each parameter for the dimensions of the matrices. Then, we depict it with the ´display()´-function. There is a possibility to change the radius $r$ again.

In [ ]:
sphere = Sphere(60, 60)
sphere.display()

After displaying the sphere, we aim at finding the metric tensor $g_{\mathrm{Sphere}}$ for it with the radius $r$.

In [ ]:
r, phi, theta = sym.symbols("r phi theta")
v_sphere = sym.Matrix([[r*sym.sin(theta)*sym.cos(phi)], 
                       [r*sym.sin(theta)*sym.sin(phi)], 
                       [r*sym.cos(theta)]])

In order to obtain the metric tensor, we have to derive this vector again with respect to $\phi$ and $\theta$, using the function ´diff(function, variable)´. 

In [ ]:
dphi = sym.diff(v_sphere, phi)
dtheta = sym.diff(v_sphere, theta)

Now, we can determine the metric tensor $g_\mathrm{Sphere}$ by the inner product variations of ´dphi´ and ´dtheta´ as previously.

In [ ]:
g_Sphere = sym.Matrix([[dphi.dot(dphi), dphi.dot(dtheta)],
                       [dtheta.dot(dphi), dtheta.dot(dtheta)]])
g_Sphere

The result can be simplified again.

In [ ]:
g_Sphere = sym.simplify(g_Sphere)
g_Sphere

<a id = 'sperepara'></a>
In the following we will examine the consequences of describing the surface with different parameterization.Therefore, we choose the cartesian coordinates $x$ and $y$: $$\left( \begin{array}{l}
	 x \\
	 y \\
	 z \\
	 \end{array}
	 \right)=\left( \begin{array}{l}
	 x \\
	 y \\
	 \pm\sqrt{r^2-x^2-y^2} \\
	 \end{array}
	 \right) \text{ for }x^2+y^2<r^2\text{ and }x,y\in\mathbb{R}\textit{.}$$ In order to describe the whole surface, we have to distinguish between the positive and the negative case. With $+$ we obtain the upper hemisphere and for $-$ the lower one. For the following calculations, we will only use the upper hemisphere. The result for the lower hemisphere is the same due to the symmetry of the sphere. We have to define the variables $x$ and $y$ and determine the derivatives.

In [ ]:
x, y = sym.symbols("x y")
v_sphere_2 = sym.Matrix([[x], [y], [sym.sqrt(r**2 - x**2 - y**2)]])
dx = sym.diff(v_sphere_2, x)
dy = sym.diff(v_sphere_2, y)

In order to find the metric tensor, one has to compute the inner products and simplify the result:

In [ ]:
g_Sphere_2 = sym.Matrix([[dx.dot(dx), dx.dot(dy)],
                         [dy.dot(dx), dy.dot(dy)]])
g_Sphere_2.simplify()
g_Sphere_2

The metric looks completely different but it describes the same surface. In the next step, we can visualize the influence of the position of measurement on the lengths on the surface. Therefore, we want to investigate what is going to happen to the arclength of a path on the sphere if we walk a certain length on the sphere in x direction. To do so, we create an array ´dx´ with ´numofinst´ instances from 0 to 0.1 with the ´linspace´-function and calculate the points for the line, which shows the way in $x$-direction saving it in ´linepoints_xdirection´. Afterwards, we can generate the line ´line_xdirection´ itself by ´k.line´ of the K3D-module.  

In [ ]:
numofinst = 10
dx = np.linspace(0, 0.1, numofinst)
linepoints_xdirection = np.array([dx, 0*dx, 
                                  0*dx]).swapaxes(0, 1).reshape(numofinst, 3)
line_xdirection = k3d.line(linepoints_xdirection, color=1, width=0.01)

<a id = 'array'></a>
Now, we want to add a line to the plot, which connects the line on the $x$-$y$-plane with the surface of the sphere. For this, we have to create an array with the values from 0 to the radius, in our case it is 1, using again the ´linspace´-function. We call this array ´dz´. Hereafter, we generate an array of points with the name ´linepoints_zdirction´ and then the line ´line_zdirection´ as before. 

In [ ]:
dz = np.linspace(0, 1, numofinst)
linepoints_zdirection = np.array([0*dz, 0*dz, np.sqrt(1. - dz**2)]
                                 ).swapaxes(0, 1).reshape(numofinst, 3)
line_zdirection = k3d.line(linepoints_zdirection, color=1, width=0.01)

To see the effect of the metric, we can now build the line on the surface of the sphere. If we project it in the $x$-$y$-plane, we would get the line ´line_xdirection´. Therefore, we are walking ´dx´ in $x$-direction, 0 in $y$-direction and in respect to the [parameterization](#sperepara), with $x$, $y$ and $z$, we are walking $\sqrt{1-0^2-dx^2}$ in $z$-direction. Then, we create the data for the line ´linepoints_surface´ with these information. Following this step, we can generate the line ´line_surface´ as we did before. 

In [ ]:
linepoints_surface = np.array([dx, 0*dx, np.sqrt(1. - dx**2)]
                              ).swapaxes(0, 1).reshape(numofinst, 3)
line_surface = k3d.line(linepoints_surface, color=1, width=0.01)

In the next step, it is necessary to add all these features to the [plot](#plotsphere) of ´plot_sphere´, creating a new sphere ´halfsphere´, with $\phi\in[0 ,\pi]$. We establish this half sphere to see the changes more clearly. To shed light on even more details, it is helpful to only display the wireframe, so we set ´wframe´ to ´True´. To be able to observe the details inside the sphere as well, we want to lower the opacity too, which we do by choosing a small value for it. 

In [ ]:
halfsphere = Sphere(60, 60, [1, np.pi])
plot_sphere = k3d.plot()
plot_sphere += line_xdirection
plot_sphere += line_zdirection
plot_sphere += line_surface
plot_sphere += halfsphere.display(wframe=True, opac=0.2)

<a id = 'wid' ></a>
To get an idea of how the length is changing on the surface, we need to be able to manipulate the plot. For this purpose, we are going to use ´ipywidgets´, which we have imported at the [beginning](#modules) of the notebook. We need to generate a function, that uses the parameter we want to change, we call it ´updateplot(x0)´. With this value, we want to move the three lines from the beginning to a place ´x0´. Therefore, we have to add ´x0´ to every value in the vector ´dx´ , substituting ´dx´ with ´dx + x0´ in ´linepoints_xdirection´ and ´linepoints_surface´. 

In order to depict the line between the ground and the surface of the sphere in $z$-direction, we have to change the upper boundary of the [array](#array), which we have used before. Because the distance from the ground to the surface of the sphere depends on the $x$-value, at the pole we have the maximal length and at the equator 0. The new upper bound has to be $\sqrt{1-x_0^2}$, so we take ´x0´ and calculate the new values, saving it in the variable ´zmax´. The ´linepoints_zdirection´ takes arrays of the length ´numofinst´, which means we have to change the format of ´x0´ to an array with all entries equal ´x0´ with ´ones(shape)´. This creates an array of length ´numofinst´, filled with ones. In the last step, it is necessary to change the attributes of the lines, and therefore reassigning the new arrays. 

To gain the possibility to actively change the plot, we have to decorate the function with the decorator ´interact´, that we imported at the [beginning](#modules). To get a slider you can use ´FloatSlider´ of the same module assigning the values between 0 and 0.9 to it. Due to the long calculations, we want to update the plot only when we release the mouse; this way, the plot is not updating while changing the position of the slider. To do this, we have to set ´continuous_updtate´ to ´False´.

In [ ]:
@interact(x0=widgets.FloatSlider(min=0, max=0.9, continuous_update=False))
def updateplot(x0):
    numofinst = 10
    linepoints_xdirection = np.array([dx + x0, 0*dx, 0*dx]
                                     ).swapaxes(0, 1).reshape(numofinst, 3)
    linepoints_surface = np.array([dx + x0, 0*dx, np.sqrt(1.-(dx + x0)**2)]
                                  ).swapaxes(0, 1).reshape(numofinst, 3)
    
    zmax = np.linspace(0, np.sqrt(1.-x0**2), numofinst)
    linepoints_zdirection = np.array([x0*np.ones(numofinst), 0*dz, zmax]
                                     ).swapaxes(0, 1).reshape(numofinst, 3)
    
    line_xdirection.vertices = linepoints_xdirection
    line_zdirection.vertices = linepoints_zdirection
    line_surface.vertices = linepoints_surface 
    
plot_sphere.display()

If you move the slider, you can see that the length of the line on the $x$-$y$-plane does not chang. Its length, instead, the line on the surface is growing towards the equator. On an Euclidean surface, it does not matter where you are measuring, on the sphere there is a difference.

<a id = 'third'></a>
## Third Example - The Ellipsoid

Afterwards, we want to investigate what is going to happen, if we use an ellipsoid with the axes $a$, $b$ and $c$ instead of a sphere using the angles $\phi\in[0,2\pi]$ and $\theta\in[0,\pi]$ as parameters. The parametrization is: $$\left( \begin{array}{l}
	 x \\
	 y \\
	 z \\
	 \end{array}
	 \right)=\left( \begin{array}{l}
	 a\cdot\sin(\theta)\cos(\phi) \\
	 b\cdot\sin(\theta)\sin(\phi) \\
	 c\cdot\cos(\theta) \\
	 \end{array}
	 \right)$$ 
As before, we will now calculate the metric.

In [ ]:
phi, theta, a, b, c = sym.symbols("phi theta a b c")
    
v_Ellipsoid = sym.Matrix([[a*sym.sin(theta)*sym.cos(phi)],
                          [b*sym.sin(theta)*sym.sin(phi)],
                          [c*sym.cos(theta)]]) 

dphi = sym.simplify(sym.diff(v_Ellipsoid, phi))
dtheta = sym.simplify(sym.diff(v_Ellipsoid, theta))

g_Ellipsoid = sym.Matrix([[dphi.dot(dphi), dphi.dot(dtheta)],
                          [dtheta.dot(dphi), dtheta.dot(dtheta)]])
g_Ellipsoid = sym.simplify(g_Ellipsoid)
g_Ellipsoid

<a id = 'ellipsoid'></a>
To visualize the impact of the metric again, we will measure the arclength of a line on the surface over-sweeping a certain angle. To do so, we define a path $\gamma$ between the starting point $p(\phi,\theta)$ and the destination by changing the angle $\phi$ by 0.2 and choosing $\theta = \frac{\pi}{2}\text{, }\forall\phi$. The path is defined by: $$\gamma(t) = \left( \begin{array}{l}
	 a\cdot\sin(\frac{\pi}{2})\cos(\phi +0.2\cdot t) \\
	 b\cdot\sin(\frac{\pi}{2})\sin(\phi +0.2\cdot t) \\
	 c\cdot\cos(\frac{\pi}{2}) \\
	 \end{array}
	 \right) = \left( \begin{array}{l}
	 a\cdot\cos(\phi +0.2\cdot t) \\
	 b\cdot\sin(\phi +0.2\cdot t) \\
	 0\\
	 \end{array}
	 \right)\text{ with }t\in [0,1]$$ To get the length of the path, we have to solve the integral: $$L(\gamma) = \int_0^1\sqrt{\frac{d\gamma}{dt}^Tg_\mathrm{Ellipsoid}(\gamma)\frac{d\gamma}{dt}}dt$$ We have to define a new variable $t$:


In [ ]:
t = sym.symbols("t")

Then we define the path $\gamma(t)$:

In [ ]:
gamma = sym.Matrix([[phi + 0.2*t],
                    [np.pi/2]])
gamma

When using the formula for the length above, it is essential to find the derivative of the path ´dgamma´. To insert the path into the metric, we can use the ´.subs(arg, substitution)´-function. This has to be done for $\theta$ and $\phi$. By transposing the matrix ´dgamma´ with ´.T´, you can simply determine the product. Then, we benefit from the square root of SymPy. The result of this product is a $1\times1$-matrix. Hence, we can transform it back to a scalar expression by taking the first and only element of this matrix. After simplifying, we display the result. 

In [ ]:
dgamma = sym.diff(gamma, t)
sym.pprint(dgamma)
metr_gamma = g_Ellipsoid.subs(phi, phi + 0.2*t).subs(theta, np.pi/2)
integrand = dgamma.T*metr_gamma*dgamma
integrand = sym.simplify(sym.sqrt(integrand[0]))
integrand


To calculate the integral, it is essential to change its form by substituting $\phi +0.2t$ with $x$. Through this, the following integral will be obtained: $$\int_{\phi}^{\phi +0.2}\sqrt{a^2\sin^2(x)+b^2\cos^2(x)}\mathrm{d}x$$ Using the Pythagorean theorem, the integrand can be expressed only in terms of the sine function:$$\int_{\phi}^{\phi +0.2}\sqrt{b^2 + (a^2-b^2)\sin^2(x)}\mathrm{d}x$$ Accordingly, the integral can be written in the following form: $$b\int_{\phi}^{\phi +0.2}\sqrt{1 - (1-\frac{a^2}{b^2})\sin^2(x)}\mathrm{d}x = b(E(\phi +0.2|1-\frac{a^2}{b^2})-E(\phi|1-\frac{a^2}{b^2}))$$ with the [incomplete elliptic integral of the second kind](https://en.wikipedia.org/wiki/Elliptic_integral) $E(\phi|k^2)$. This [function](#modules) can be found in the ´scipy.special´-module. To calculate the integral, we are going to define the function ´arclength(a, b, phi)´ that takes the axes and the angle $phi$ as arguments.

In [ ]:
def arclength(a,b,phi):
    k_sq = 1-(a/b)**2
    return b*(special.ellipeinc(phi + 0.2, k_sq) - special.ellipeinc(phi, k_sq))

Now, we can calculate the length for ´phi = 0´, ´a = 1´ and ´b = 2´.

In [ ]:
length = arclength(1, 2, 0)
length

The plot is exactly the same as we did [before](#example), only with the constants $a$, $b$ and $c$ for the axes. Thus, we assign a list of these three constants to the class, which are set to ´a = 1´, ´b = 2´ and ´c = 1´ by default. For the plot, we have to choose values for them, in this case 1, 2 and 1. You can try again to change the values of the axes.

In [ ]:
class Ellipsoid(Surface2D):
    def __init__(self, dim1, dim2, constants=[1, 2, 1]):
        super().__init__(0, np.pi, dim1, 0, 2*np.pi, dim2, constants)

    def get_points(self):
        a, b, c = self.constants
        theta, phi = self.parameters.T
        self.points = np.array([a*np.sin(theta)*np.cos(phi),
                                b*np.sin(theta)*np.sin(phi),
                                c*np.cos(theta)], 
                               dtype=np.float32).T
        
ellipsoid = Ellipsoid(60, 60)
ellipsoid.display()

Afterwards, we plot the line ´linepoints´ between the two points, as [described](#ellipsoid) before dividing the length 0.2 in ´dimt = 10´ instances.

In [ ]:
dimt = 10
a = 1
b = 2
t = np.linspace(0, 1, dimt)
linepoints = np.array([a*np.cos(0.2*t),
                       b*np.sin(0.2*t),
                       0*t]).T
line_ellipsoid = k3d.line(linepoints, color=1, width=0.01)

Here, we plot again our objects.

In [ ]:
plot_ellipsoid = k3d.plot()
plot_ellipsoid += ellipsoid.display()
plot_ellipsoid += line_ellipsoid

In the last step, we want to do the same as we did for the [sphere](#wid), defining our parameter ´phi0´, which we can manipulate with the decorator between $-\frac{\pi}{2}$ and $\frac{\pi}{2}$. Afterwards, the computer can calculate the new length and print it, in order to show its changes. Because we are only looking at the $x$-$y$-plane, the $z$-value remains 0, so we have only variations in the first two matrix entries of our ´linepoints´. Actually, we have to multiply ´phi0´ again with a vector of ones that has the same dimension as $t$, like in the example of the sphere. Hence, we create this array with ´ones(shape)´. Hereafter, we can update the position of the line, referring to the attribute ´vertices´ of the line. After the function, the plot is displayed.  

In [ ]:
@interact(phi0=widgets.FloatSlider(min=-np.pi/2, max=np.pi/2, continuous_update=False))
def updateplot(phi0):
    phi0 = phi0
    length = arclength(a, b, phi0)
    print('length = ', length)
    linepoints = np.array([a*np.cos(phi0*np.ones(dimt) + 0.2*t), 
                           b*np.sin(phi0*np.ones(dimt) + 0.2*t), 0*t]
                          ).swapaxes(0, 1).reshape(dimt, 3)
    line_ellipsoid.vertices = linepoints
    
plot_ellipsoid.display()

Now, you can try what is going to happen, if you investigate the arclength on the surface with a constant opening angle of $\phi = 0.2$ at different places. 

<a id = 'forth'></a>
## Forth Example - Gnocchi

At the end, we want to repeat the procedure for a parameterized pasta surface, from the book "Pasta by Design" by George L. Legendre. This is to show the possibilities to also parameterize nontrivial surfaces and do the same calculations, seen before. 

The surface is parametrized within $i\in[0,1.3\pi]$ and $j\in[0,\pi]$. 
$$
\left( \begin{array}{l}
	 x \\
	 y\\
	 z\\
	 \end{array}
	 \right)=
\left( \begin{array}{l}
\sin(j)\cos(i)\cdot(0.2|\sin(5j)|+\frac{i}{\pi})\\
\sin(j)\sin(i)\cdot(0.2|\sin(5j)|+\frac{i}{\pi})\\
	 1.5\cos(j)\\
	 \end{array}
	 \right)
$$

In [ ]:
class Gnocchi(Surface2D):
    def __init__(self, p1pts, p2pts):
        super().__init__(0, 1.3*np.pi, p1pts, 0, np.pi, p2pts)
        
    def get_points(self):
        i, j = self.parameters.T
        self.points = np.array([np.sin(j)*np.cos(i)*(0.2*abs(np.sin(5*j))+i/np.pi),
                                np.sin(j)*np.sin(i)*(0.2*abs(np.sin(5*j))+i/np.pi),
                                1.5*np.cos(j)], 
                               dtype=np.float32).T
        
gnocchi = Gnocchi(50, 150)
gnocchi.display()

Consequently, we will compute the metric-tensor $g_\mathrm{Gnocchi}$ for this nontrivial surface. The result is demonstrating the complexity of this surface. It proofs in some way, that it is possible to model nearly every surface in a proper way and do geometric calculations on it. The SymPy-module offers a huge amount of useful functions, which help making these calculations for the user.

In [ ]:
i, j = sym.symbols("i j")
    
v_Gnocchi = sym.Matrix([sym.sin(j)*sym.cos(i)*(0.2*abs(sym.sin(5*j)) + i/sym.pi),
                       sym.sin(j)*sym.sin(i)*(0.2*abs(sym.sin(5*j)) + i/sym.pi),
                       1.5*sym.cos(j)])

di = sym.simplify(sym.diff(v_Gnocchi, i))
dj = sym.simplify(sym.diff(v_Gnocchi, j))

g_Gnocchi = sym.Matrix([[di.dot(di), di.dot(dj)],
                        [dj.dot(di), dj.dot(dj)]])
g_Gnocchi = sym.simplify(g_Gnocchi)
g_Gnocchi